In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Tue Jan  2 10:36:36 2018

@author: tjian
"""
from keras.layers import LSTM
from keras.models import Sequential



# (unit + dim)*unit*4 + unit*4
# (unit + dim + 1)*unit*4
# 多lstm是将每个前面的所有输出再放到输入里来

model = Sequential() 
model.add(LSTM(32,input_shape=(1,3)))
# input_shape 1-> time step 
#             3-> data dim
# 参数=4608   (32+3)*(32*4) + 32*4 4=控制z,z_in,z_out,z_forget

model = Sequential() 
model.add(LSTM(32,input_shape=(1,1)))
# input_shape 1-> time step 
#             1-> data dim
# 参数=4532   (32+1)*(32*4) + 32*4  第一个4=控制z,z_in,z_out,z_forget
# 第二个 4个bias

# 测试李宏毅38讲 RNN的课件
from keras.layers import LSTM
from keras.models import Sequential
import numpy as np

x = np.array([[1,0,0],[3,1,0],[2,0,0],[4,1,0],[2,0,0],[1,0,1],[3,-1,0],[6,1,0],[1,0,1]])
y = np.array([0,0,0,0,0,7,0,0,6])

model = Sequential()
model.add(LSTM(1, activation=None, stateful=True, batch_input_shape=(1,1,3)))
xxx = model.get_weights()
xxx[0]=np.array([[0,0,1,0],[100,100,0,0],[0,0,0,100]])
xxx[1]=np.array([[1,1,1,1]]) # 这个是h_t-1输出的参数的w 可以使用 model.layers[0].re
xxx[2]=np.array([-10,10,0,-10])
model.set_weights(xxx)


print model.predict_on_batch(np.array([1,0,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([3,1,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([2,0,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([4,1,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([2,0,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([1,0,1]).reshape(-1,1,3))
print model.predict_on_batch(np.array([3,-1,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([6,1,0]).reshape(-1,1,3))
print model.predict_on_batch(np.array([1,0,1]).reshape(-1,1,3))


from keras import backend as K
K.eval(model.layers[0].states[0]) # h值
K.eval(model.layers[0].states[1]) # c值

print('h={},c={}'.format(K.eval(model.layers[0].states[0]),K.eval(model.layers[0].states[1])))

model.reset_states() # 将 c值设置为0

# 1. h=[[ 0.]],c=[[ 0.]]
# 2. h=[[ 0.]],c=[[ 3.]]
# 3. h=[[ 0.]],c=[[ 3.]]
# 4. h=[[ 0.]],c=[[ 7.]]
# 5. h=[[ 0.]],c=[[ 7.]]
# 6. h=[[ 7.]],c=[[ 7.]]
# 7. h=[[ 0.]],c=[[ 0.]]
# 8. h=[[ 0.]],c=[[ 6.]]
# 9. h=[[ 6.]],c=[[ 6.]]


# 上面实现的另一种方法，更能很好地理解LSTM
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
import numpy as np

x = np.array([[1,0,0],[3,1,0],[2,0,0],[4,1,0],[2,0,0],[1,0,1],[3,-1,0],[6,1,0],[1,0,1]])
'''
http://blog.csdn.net/u011327333/article/details/78501054
inputs1 = Input(shape=(9, 3))
lstm1, state_h, state_c = LSTM(1, activation=None,return_sequences=True,return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])

xxx = model.get_weights()
xxx[0]=np.array([[0,0,1,0],[100,100,0,0],[0,0,0,100]])
xxx[1]=np.array([[1,1,1,1]]) # 这个是h_t-1输出的参数的w 可以使用 model.layers[0].re
xxx[2]=np.array([-10,10,0,-10])
model.set_weights(xxx)

model.predict(x.reshape(-1,9,3))
'''

Using TensorFlow backend.


[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[0.]]
[[7.]]
[[0.]]
[[0.]]
[[6.]]
h=[[6.]],c=[[6.]]


'\nhttp://blog.csdn.net/u011327333/article/details/78501054\ninputs1 = Input(shape=(9, 3))\nlstm1, state_h, state_c = LSTM(1, activation=None,return_sequences=True,return_state=True)(inputs1)\nmodel = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])\n\nxxx = model.get_weights()\nxxx[0]=np.array([[0,0,1,0],[100,100,0,0],[0,0,0,100]])\nxxx[1]=np.array([[1,1,1,1]]) # \xe8\xbf\x99\xe4\xb8\xaa\xe6\x98\xafh_t-1\xe8\xbe\x93\xe5\x87\xba\xe7\x9a\x84\xe5\x8f\x82\xe6\x95\xb0\xe7\x9a\x84w \xe5\x8f\xaf\xe4\xbb\xa5\xe4\xbd\xbf\xe7\x94\xa8 model.layers[0].re\nxxx[2]=np.array([-10,10,0,-10])\nmodel.set_weights(xxx)\n\nmodel.predict(x.reshape(-1,9,3))\n'

理解LSTM：

In [4]:
from keras.layers import LSTM, Lambda,Input
from keras.models import Model, Sequential
import numpy as np

np.random.seed(7) # 可重复weights随机值
'''
 0.39346123,  0.41240287, -0.38993686, -0.07691038]], dtype=float32),
 array([[-0.52859539, -0.79616123, -0.02723205,  0.29321092]], dtype=float32),
 array([ 0.,  1.,  0.,  0.], dtype=float32)]
'''
x = np.array([1,0,0])
inputss = Input(shape=(3,1))
lstm, state_h, state_c = LSTM(1,return_sequences=True,recurrent_activation='sigmoid',return_state=True)(inputss)
#lstm = LSTM(1,return_sequences=True, recurrent_activation='sigmoid')(inputss) # LSTM缺省是hard_sigmoid
# 研究：http://blog.csdn.net/silent56_th/article/details/73442391
model = Model(inputs = inputss, outputs=[lstm,state_h,state_c])
xxx = model.get_weights()
xxx[2]=np.array([0.,0.,0.,0.])
model.set_weights(xxx)
x1=xxx[0]
x2=xxx[1]
xx=np.vstack((x2,x1))

def sigmoid(x):
    return 1/(1+np.exp(-x))

def tanh_h(x):
    return (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))

i_t = sigmoid(np.dot(input1.reshape(1,2), xx[:,0].reshape(2,1)))
f_t = sigmoid(np.dot(input1.reshape(1,2), xx[:,1].reshape(2,1)))
c_t = tanh_h(np.dot(input1.reshape(1,2), xx[:,2].reshape(2,1)))
o_t = sigmoid(np.dot(input1.reshape(1,2), xx[:,3].reshape(2,1)))

c_tt = i_t*c_t
h_t = o_t*tanh_h(c_tt)

[array([[ 0.99018717,  1.0247939 , -0.37382555, -0.9029063 ]],
       dtype=float32),
 array([[-0.53536123, -0.42826614, -0.68243545, -0.25349215]],
       dtype=float32),
 array([0., 1., 0., 0.], dtype=float32)]

xxx【0】是控制本cell的4个weihgt参数，依次是：
$Gate_{in}, Gate_{foget},Contron_{in},Gate_{out}$ 

xxx【1】是控制上一个cell到本cell的4个weight参数，顺序同上

xxx【2】是控制本cell的4个bias，顺序同上

算法见https://www.jianshu.com/p/dcec3f07d3b5

![avatar](img/lstm-1.png)
![avatar](img/lstm-2.png)
![avatar](img/lstm-3.png)

如果是shape=（3，1）， timestep=3， input_dim=1:
则需要学习：$[W_{ih},W_{ix}],[W_{fh},W_{fx}],[W_{ch},W_{cx}],[W_{oh},W_{ox}], 加上 bias=4，共8+4=12组参数 , 输入=[h_{t-1},x_t]$

如果是shape=（1，3）， timestep=1， input_dim=3:
则需要学习：$[W_{ih},W_{ix}-1,W_{ix}-2,W_{ix}-3],[W_{fh},W_{fx}-1,W_{fx}-2,W_{fx}-3],[W_{ch},W_{cx}-1,W_{cx}-2,W_{cx}-3],[W_{oh},W_{ox}-1,W_{ox}-2,W_{ox}-3], 加上 bias=4，共16+4=20组参数, 输入=[h_{t-1},x_t-1,x_t-2,x_t-3]$

如果是shape=(3,1), timestep=3， input_dim=1, lstm=2:
则需要学习：$[W_{ih}-1,W_{ih}-2,W_{ix}],[W_{fh}-1,W_{fh}-2,W_{fx}],[W_{ch}-1,W_{ch}-2,,W_{cx}],[W_{oh}-1,W_{oh}-2,W_{ox}], 加上 bias=4，一个LSTM有12+4=16组参数 , 两个有16x2=32组参数，输入=[h_{t-1}-1,h_{t-1}-2,x_t]$

如果是shape=(1,3), timestep=1， input_dim=3, lstm=2: 则需要学习：
则需要学习：$[W_{ih}-1,W_{ih}-2,W_{ix}-1,W_{ix}-2,W_{ix}-3],[W_{fh}-1,W_{fh}-2,W_{fx}-1,W_{fx}-2,W_{fx}-3],[W_{ch}-1,W_{ch}-2,W_{cx}-1,W_{cx}-2,W_{cx}-3],[W_{oh}-1,W_{oh}-2,W_{ox}-1,W_{ox}-2,W_{ox}-3], 加上 bias=4，一个LSTM有20+4=24组参数，两个LSTM则有 24x2=48组参数 输入=[h_{t-1}-1, h_{t-1}-2,x_t-1,x_t-2,x_t-3]$


In [ ]:
inputs1 = Input(shape=(9, 3))
lstm1 = LSTM(1, activation=None,return_sequences=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1])

model.compile(loss='mse',optimizer='adam')
model.fit(x.reshape(-1,9,3),y.reshape(1,9,1),epochs=500,batch_size=1,verbose=2)


# 参数个数：（x_times+x_dim)*unit + 4*unit^2 + 4*unit
# 分别是： xx[0], xx[1], xx[2]
# 即： kernal的weight， recurent的weight, 和 bias
'''
model = Sequential()
model.add(LSTM(1,activation=None, return_sequences=True,input_shape=(9,3)))
model.compile(loss='mse',optimizer='adam')
model.fit(x.reshape(-1,9,3),y.reshape(1,9,1),epochs=500,batch_size=1,verbose=2)

'''
# 38讲RNN训练参数

from keras.layers import LSTM
from keras.models import Sequential
import numpy as np
from keras.callbacks import EarlyStopping

x = np.array([[1,0,0],[3,1,0],[2,0,0],[4,1,0],[2,0,0],[1,0,1],[3,-1,0],[6,1,0],[1,0,1]])
y = np.array([0,0,0,0,0,7,0,0,6])

model = Sequential()
model.add(LSTM(1, activation=None, stateful=True, batch_input_shape=(1,1,3)))


model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
xx = x.reshape(-1,1,3)
yy = y.reshape(-1,1)

'''
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

for i in range(300):
	model.fit(xx, yy, epochs=1, batch_size=1, shuffle=False, callbacks=[early_stopping])
	model.reset_states()
    
# model.fit(xx,yy,batch_size=1,epochs=5032,shuffle=False)
'''

for i in range(1000):
    for x_i,y_i in zip(x,y):
        x_i = x_i.reshape(-1,1,3)
        y_i = y_i.reshape(1,1)
        model.train_on_batch(x_i, y_i)
    model.reset_states()
    if(i%50==0):
        loss_and_metric = model.evaluate(xx, yy, batch_size=1)    
        print loss_and_metric

# loss_and_metrics = model.evaluate(xx, yy, batch_size=1)
# classes = model.predict_classes(xx, batch_size=1)
# pred = model.predict_proba(xx, batch_size=1)
        

# 例子研究
        
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from numpy import array
# define model
inputs1 = Input(shape=(3, 1))
lstm1, state_h, state_c = LSTM(1, return_state=True)(inputs1)
model = Model(inputs=inputs1, outputs=[lstm1, state_h, state_c])
# define input data
data = array([0.1, 0.2, 0.3]).reshape((1,3,1))
# make and show prediction
print(model.predict(data))


# 例子研究
# http://www.guanggua.com/question/43237170-keras-stateful-lstm.html

from keras.layers import LSTM, Dense
from keras.models import Sequential
import numpy as np

nb_samples = 100000
X = np.random.randn(nb_samples)
Y = X[1:]
X = X[:-1]
X = X.reshape((len(Y), 1, 1))
Y = Y.reshape((len(Y), 1))

model = Sequential()
model.add(LSTM(batch_input_shape=(1, 1, 1),
               units =10,
               activation='tanh', stateful=True
          )
    )
model.add(Dense(units=1, activation=None))
model.compile(loss='mse', optimizer='adam')

for epoch in range(10000):
    model.reset_states()
    train_loss = 0
    for i in range(Y.shape[0]):
        train_loss += model.train_on_batch(X[i:i+1],
                         Y[i:i+1],
                         )
    print '# epoch', epoch, '  loss ', train_loss/float(Y.shape[0])
    
    
    
# 研究SimpleRNN, SimpleRUUCell, RNN
from keras.layers import RNN,SimpleRNN,SimpleRNNCell
from keras.models import Sequential
import numpy as np

model = Sequential()
model.add(SimpleRNN(1, input_shape=(3,1), activation=None))

xx = model.get_weights()
xx[0]=np.array([[1.]]) # x输入的W
xx[1]=np.array([[2.]]) # cell 的W
xx[2]=np.array([0]) # x输入的bias

model.set_weights(xx)

model.predict(np.array([1,1,1]).reshape(-1,3,1)) # =7
# 1-> cell=1, out=1
# 2-> cell=w*前cell=2*1 = 2 + x*1 = 3, out=3
# 3-> cell=2*3+1=7, out = 7

model.predict(np.array([1,2,3]).reshape(-1,3,1)) # =15
# 1-> cell=1, out=1
# 2-> cell=w*前cell=2*1  + 1*2 = 4, out=4
# 3-> cell=2*4+1*3=11, out = 11

xx[0]=np.array([[2.]]) # x输入的W
xx[1]=np.array([[1.]]) # cell 的W
xx[2]=np.array([0]) # x输入的bias
model.set_weights(xx)

model.predict(np.array([1,1,1]).reshape(-1,3,1)) # =7
# 1-> cell=1*0+2*1, out=2
# 2-> cell=w*前cell=1*2+ 2*1 = 4, out=4
# 3-> cell=1*4+2*1=6, out = 6

model.predict(np.array([1,2,3]).reshape(-1,3,1)) # =11
# 1-> cell=1*0+2*1, out=2
# 2-> cell=w*前cell=1*2+ 2*2 = 6, out=6
# 3-> cell=1*6+2*3=12, out = 12


# http://blog.csdn.net/u013518890/article/details/74938758
# LSTM with Variable Length Input Sequences to One Character Output
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
from theano.tensor.shared_randomstreams import RandomStreams
# fix random seed for reproducibility
numpy.random.seed(7)
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))
# prepare the dataset of input to output pairs encoded as integers
num_inputs = 1000
max_len = 5
dataX = []
dataY = []
for i in range(num_inputs):
    start = numpy.random.randint(len(alphabet)-2)
    end = numpy.random.randint(start, min(start+max_len,len(alphabet)-1))
    sequence_in = alphabet[start:end+1]
    sequence_out = alphabet[end + 1]
    dataX.append([char_to_int[char] for char in sequence_in])
    dataY.append(char_to_int[sequence_out])
    print sequence_in, '->', sequence_out
# convert list of lists to array and pad sequences if needed
X = pad_sequences(dataX, maxlen=max_len, dtype='float32')
# reshape X to be [samples, time steps, features]
X = numpy.reshape(X, (X.shape[0], max_len, 1))
# normalize
X = X / float(len(alphabet))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# create and fit the model
batch_size = 1
model = Sequential()
model.add(LSTM(32, input_shape=(X.shape[1], 1)))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=500, batch_size=batch_size, verbose=2)
# summarize performance of the model
scores = model.evaluate(X, y, verbose=0)
print("Model Accuracy: %.2f%%" % (scores[1]*100))
# demonstrate some model predictions
for i in range(20):
    pattern_index = numpy.random.randint(len(dataX))
    pattern = dataX[pattern_index]
    x = pad_sequences([pattern], maxlen=max_len, dtype='float32')
    x = numpy.reshape(x, (1, max_len, 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    print seq_in, "->", result
    
    
# lstm mnist实现：97%

import cPickle as pickle
import np_utils
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np


def read_data(data_file):  
    import gzip  
    f = gzip.open(data_file, "rb")  
    train, val, test = pickle.load(f)  
    f.close()  
    train_x = train[0]  
    train_y = train[1]  
    test_x = test[0]  
    test_y = test[1]  
    return train_x, train_y, test_x, test_y

train_x, train_y, test_x, test_y = read_data('../dataset/mnist.pkl.gz')
x_train = train_x.reshape(-1,28,28)
x_test = test_x.reshape(-1,28,28)

y_train = np_utils.to_categorical(train_y,10)
y_test = np_utils.to_categorical(test_y,10)

model = Sequential()
model.add(LSTM(32,input_shape=(28,28)))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

model.fit(x_train,y_train)
model.evaluate(x_test,y_test)